In [3]:
import numpy as np
import pandas as pd

**4.1 Hoàn thiện function create_train_dataset() để tổ chức dữ liệu bảng 1 vào array 2 chiều như bên dưới.**

In [4]:
######################### # Create data #########################
# import numpy as np
def create_train_data():
    #
    pd_data = pd.read_csv("data/play_tennis.csv")
    pd_data = pd_data.drop(columns=["Day"])
    data = pd_data.values.tolist()
    return np.array(data)


train_data = create_train_data()
print(train_data)

[['Sunny' 'Hot' 'High' 'Weak' 'no']
 ['Sunny' 'Hot' 'High' 'Strong' 'no']
 ['Overcast' 'Hot' 'High' 'Weak' 'yes']
 ['Rain' 'Mild' 'High' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'no']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'yes']
 ['Overcast' 'Mild' 'High' 'Weak' 'no']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'yes']]


**4.2 Hoàn thiện function compute_prior_probability tính P("Play Tennis" = "Yes") and tính P("Play Tennis" = "No") như bên dưới:**

In [5]:
def compute_prior_probablity(train_data):
    y_unique = ["no", "yes"]
    prior_probability = np.zeros(len(y_unique))

    #
    for i, y in enumerate(y_unique):
        prior_probability[i] = float(np.where(train_data[:, -1] == y, 1, 0).mean())
    return prior_probability


prior_probablity = compute_prior_probablity(train_data)
print("P(play tennis = No", prior_probablity[0])
print("P(play tennis = Yes", prior_probablity[1])

P(play tennis = No 0.4
P(play tennis = Yes 0.6


In [6]:
# 14 -> a)

**4.3 Hoàn thiện function compute_conditional_probability để tính likelihood (The probability of "A" being True. Given "B" True, P(A|B)) như bên dưới:**

In [7]:
def compute_conditional_probability(train_data):
    y_unique = ["no", "yes"]
    conditional_probability = []
    list_x_name = []
    for i in range(0, train_data.shape[1] - 1):
        x_unique = np.unique(train_data[:, i])
        list_x_name.append(x_unique)

        x_conditional_probability = np.zeros((len(y_unique), len(x_unique)))
        for j in range(0, len(y_unique)):
            for k in range(0, len(x_unique)):
                x_conditional_probability[j, k] = len(
                    np.where(
                        (train_data[:, i] == x_unique[k])
                        & (train_data[:, 4] == y_unique[j])
                    )[0]
                ) / len(np.where(train_data[:, 4] == y_unique[j])[0])

        # for feature i, label j, feature_name x1
        conditional_probability.append(x_conditional_probability)

    return conditional_probability, list_x_name

In [8]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)
print("x1 = ", list_x_name[0])
print("x2 = ", list_x_name[1])
print("x3 = ", list_x_name[2])
print("x4 = ", list_x_name[3])

print("P(x1|play tennis = No) = ", conditional_probability[0])
print("P(x1|play tennis = Yes) = ", conditional_probability[1])

x1 =  ['Overcast' 'Rain' 'Sunny']
x2 =  ['Cool' 'Hot' 'Mild']
x3 =  ['High' 'Normal']
x4 =  ['Strong' 'Weak']
P(x1|play tennis = No) =  [[0.25       0.25       0.5       ]
 [0.33333333 0.5        0.16666667]]
P(x1|play tennis = Yes) =  [[0.25       0.5        0.25      ]
 [0.5        0.16666667 0.33333333]]


In [9]:
# 15 -> b)

**4.4 Hoàn thiện function get_index_from_value để tính trả về index tương ứng với feature name:**

In [10]:
# This function is used to return the index of the feature name
def get_index_from_value(feature_name, list_features):
    return np.where(list_features == feature_name)[0][0]

**Câu hỏi 16: Hãy cho biết kết quả của đoạn chương trình sau đây:**

In [11]:
train_data = create_train_data()
_, list_x_name = compute_conditional_probability(train_data)
outlook = list_x_name[0]

i1 = get_index_from_value("Overcast", outlook)
i2 = get_index_from_value("Rain", outlook)
i3 = get_index_from_value("Sunny", outlook)
print(i1, i2, i3)

0 1 2


In [12]:
# 16 -> c)

**Câu hỏi 17: Hãy cho biết kết quả của đoạn chương trình sau đây:**

In [15]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)

# Compute P("Outlook"="Sunny"|Play Tennis"="Yes")
x1 = get_index_from_value("Sunny", list_x_name[0])
print(
    "P('Outlook'='Sunny'|Play Tennis'='Yes') = ",
    np.round(conditional_probability[0][1, x1], 2),
)

P('Outlook'='Sunny'|Play Tennis'='Yes') =  0.17


In [16]:
# 17 -> d)

**Câu hỏi 18: Hãy cho biết kết quả của đoạn chương trình sau đây:**

In [18]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)
# Compute P("Outlook"="Sunny"|Play Tennis"="No")
x1 = get_index_from_value("Sunny", list_x_name[0])
print(
    "P('Outlook'='Sunny'|Play Tennis'='No') = ",
    np.round(conditional_probability[0][0, x1], 2),
)

P('Outlook'='Sunny'|Play Tennis'='No') =  0.5


In [19]:
# 18 -> a)

**4.5 Hoàn thiện function train_naive_bayes như bên dưới:**

In [22]:
###########################
# Train Naive Bayes Model
###########################
def train_naive_bayes(train_data):
    # Step 1: Calculate Prior Probability
    y_unique = ["no", "yes"]
    prior_probability = compute_prior_probablity(train_data)

    # Step 2: Calculate Conditional Probability
    conditional_probability, list_x_name = compute_conditional_probability(train_data)
    return prior_probability, conditional_probability, list_x_name

**4.6 Hoàn thiện function prediction_play_tennis để hỗ trợ AD có nên đi chơi tennis vào ngày D11 không:**

In [20]:
#################### # Prediction ####################
def prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability):

    x1 = get_index_from_value(X[0], list_x_name[0])
    x2 = get_index_from_value(X[1], list_x_name[1])
    x3 = get_index_from_value(X[2], list_x_name[2])
    x4 = get_index_from_value(X[3], list_x_name[3])

    p0 = (
        prior_probability[0]
        * conditional_probability[0][0, x1]
        * conditional_probability[1][0, x2]
        * conditional_probability[2][0, x3]
        * conditional_probability[3][0, x4]
    )

    p1 = (
        prior_probability[1]
        * conditional_probability[0][1, x1]
        * conditional_probability[1][1, x2]
        * conditional_probability[2][1, x3]
        * conditional_probability[3][1, x4]
    )

    # your code here ***********************

    if p0 > p1:
        y_pred = 0
    else:
        y_pred = 1
    return y_pred

**Câu hỏi 19: Hãy cho biết kết quả của đoạn chương trình sau đây:**

In [23]:
X = ["Sunny", "Cool", "High", "Strong"]
data = create_train_data()
prior_probability, conditional_probability, list_x_name = train_naive_bayes(data)
pred = prediction_play_tennis(
    X, list_x_name, prior_probability, conditional_probability
)

if pred:
    print("Ad should go!")
else:
    print("Ad should not go!")

Ad should not go!


In [ ]:
# 19 -> a)